In [2]:
import pickle

import tflearn
import tensorflow as tf
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import json
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
stemmer =LancasterStemmer()

with open('intents.json', 'r', encoding='utf8') as file:
    data = json.load(file)

In [5]:
words = []
labels = []
docs_x = []
docs_y = []
ignore_wrods = ['?', '.', ',', '\'', ';' ':']
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_wrods]

words = sorted(list(set(words)))
labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [6]:
training = np.array(training)
output = np.array(output)

In [7]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)


model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.67277 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.67277 - acc: 0.9382 -- iter: 40/41
Training Step: 6000  | total loss: 0.63127 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.63127 - acc: 0.9444 -- iter: 41/41
--


In [8]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

In [27]:
inp = "Life at VIT"

results = model.predict([bag_of_words(inp, words)])
results_index = np.argmax(results)
tag = labels[results_index]

for tg in data["intents"]:
    if tg['tag'] == tag:
        responses = tg['responses']

print(random.choice(responses))

Life at VIT is not gigantically different from life at colleges at par with it, like MIT, COEP, PICT etc. Life here revolves mainly around three things - exams, fests and extra/co-ciricular activities. Lectures and college coaxing being constant ;-).
Since the campus isnt enormous by relative standards, you get to see your friends from other departments on a regular basis. Friend groups formed in first year largely stay intact till the last year.
Every sem has five stressful times and two stressbusters. Class tests, midsems, end sem submissions, end sems and ofcourse results constitute the stressful times. First sem has Vishwakarandak and Diwali Break while second Sem has Melange and Vishwotsav.FY & SY students get to attend FE welcome in first sem while TY and B.Tech students attend Farewell. Also there is a plethora of other interesting events going on over the year such as Vishwaprenuer, TEDxVITPune etc.
Doing something apart from attending (or not attending) lectures is highly reco